In [1]:
import pandas as pd
import numpy as np
df=pd.read_csv('updated_coughvid.csv')

In [2]:
df

,rmse,chroma_stft,spec_cent,spec_bw,rolloff,zcr,mfcc{1},mfcc{2},mfcc{3},mfcc{4},...,rapJitter,ppq5Jitter,ddpJitter,localShimmer,localdbShimmer,apq3Shimmer,aqpq5Shimmer,apq11Shimmer,ddaShimmer,label
0,0.065850,0.403767,1316.806414,1373.998076,2637.860622,0.057043,-396.59204,69.540160,2.152846,13.354017,...,0.021518,0.024053,0.064553,0.187378,1.718899,0.089158,0.143649,0.352439,0.267473,0
1,0.033997,0.532892,2474.234037,2125.162327,4869.731365,0.186172,-435.21085,45.288998,-12.166409,10.258451,...,0.014014,0.018379,0.042043,0.130333,1.313323,0.049385,0.059807,0.110768,0.148156,1
2,0.023310,0.370873,2158.381678,2007.817231,4750.294555,0.125032,-412.62552,54.555480,-1.768253,3.977824,...,0.033526,0.037981,0.100577,0.272967,2.125802,0.132889,0.196880,0.160610,0.398668,1
3,0.047035,0.479319,2678.491315,2139.232294,5136.450596,0.246256,-393.00226,48.030190,-29.901045,25.478853,...,0.032028,0.031223,0.096085,0.175612,1.617597,0.070595,0.094968,0.154762,0.211784,0
4,0.011785,0.741700,3316.010424,2345.969321,6007.887783,0.278178,-556.18726,10.974088,-10.207122,6.857212,...,0.023268,0.026566,0.069803,0.138516,1.320449,0.075879,0.081973,0.145956,0.227636,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24437,0.030827,0.429203,2974.741815,2265.905377,5435.183318,0.216093,-424.03302,38.712093,-16.247238,12.712377,...,0.029542,0.036789,0.088625,0.202130,1.619779,0.075903,0.199575,0.160610,0.227709,1
24438,0.030711,0.535591,2719.621677,2132.117936,5260.719083,0.198633,-471.09518,28.604359,-8.991700,12.397835,...,0.019939,0.022426,0.059816,0.176616,1.347352,0.107413,0.148694,0.117848,0.322239,1
24439,0.159654,0.389324,2360.664509,1696.391140,4114.219514,0.169187,-206.32933,59.016940,-74.789270,-1.210189,...,0.019389,0.020776,0.058168,0.164419,1.429379,0.093083,0.119234,0.175221,0.279249,0
24440,0.030981,0.451451,2788.705294,1800.585083,4708.568653,0.239603,-469.87784,59.399067,-41.076590,-2.028796,...,0.032847,0.036794,0.098542,0.205180,1.715662,0.105993,0.139264,0.166415,0.317979,1


In [3]:
df = pd.DataFrame(df)

In [4]:

from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout,BatchNormalization
from sklearn.model_selection import train_test_split



# Separate file names and labels from features
filenames = df.iloc[:, 0]
labels = df.iloc[:, -1]
features = df.iloc[:, 1:-1]




In [5]:
# Normalize features
scaler = StandardScaler()
features = scaler.fit_transform(features)


In [6]:
# Prepare the data
sequence_length = 10
n_features = features.shape[1]
sequences = []
targets = []


In [7]:
for i in range(sequence_length, len(df)-1):
    sequences.append(features[i-sequence_length:i])
    targets.append(labels[i-1])

X = np.array(sequences)
y = np.array(targets)

In [8]:
X = np.array(sequences)
y = np.array(targets)

In [9]:
# Slice filenames to match the number of samples in X and y
filenames = filenames[sequence_length:-1]

In [10]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test, filenames_train, filenames_test = train_test_split(X, y, filenames, test_size=0.2, random_state=42)


In [11]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(32, input_shape=(sequence_length, n_features)))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(Dense(1, activation='relu'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy}")

# Make predictions
predictions = model.predict(X_test)

# Save predictions and corresponding file names to a CSV file
results = pd.DataFrame({'filename': filenames_test, 'label': y_test, 'prediction': predictions.flatten()})
results.to_csv('predictions.csv', index=False)

Epoch 1/50
1222/1222 [==============================] - 13s 6ms/step - loss: 3.9267 - accuracy: 0.6717 - val_loss: 3.2963 - val_accuracy: 0.7252
Epoch 2/50
1222/1222 [==============================] - 7s 6ms/step - loss: 3.2688 - accuracy: 0.7258 - val_loss: 3.3721 - val_accuracy: 0.7254
Epoch 3/50
1222/1222 [==============================] - 8s 7ms/step - loss: 3.2092 - accuracy: 0.7319 - val_loss: 2.7607 - val_accuracy: 0.7684
Epoch 4/50
1222/1222 [==============================] - 11s 9ms/step - loss: 2.9995 - accuracy: 0.7479 - val_loss: 2.6514 - val_accuracy: 0.7800
Epoch 5/50
1222/1222 [==============================] - 11s 9ms/step - loss: 2.8302 - accuracy: 0.7483 - val_loss: 2.4258 - val_accuracy: 0.7835
Epoch 6/50
1222/1222 [==============================] - 9s 7ms/step - loss: 2.8642 - accuracy: 0.7512 - val_loss: 2.6487 - val_accuracy: 0.7763
Epoch 7/50
1222/1222 [==============================] - 8s 7ms/step - loss: 2.6496 - accuracy: 0.7693 - val_loss: 2.6911 - val_accura

In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
# Evaluate the model on test data
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)

# Calculate metrics
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn+fp)

# Print results
print(f"Test loss: {loss}")
print(f"Test accuracy: {accuracy}")
print(f"Test precision: {precision}")
print(f"Test recall: {recall}")
print(f"Test F1 score: {f1}")
print(f"Test ROC AUC: {auc}")
print(f"Test specificity: {specificity}")

153/153 [==============================] - 1s 3ms/step
Test loss: 0.3941638767719269
Test accuracy: 0.857990562915802
Test precision: 0.9320096269554753
Test recall: 0.7272300469483568
Test F1 score: 0.8169831223628691
Test ROC AUC: 0.8431217336664163
Test specificity: 0.9590134203844759


In [16]:

#model.save('coswara_LSTM.h5')

# import pickle
# import joblib
# with open('coswara_LSTM.h5', 'wb') as f:
#     # Use pickle to dump the dictionary into the file
#     pickle.dump(model, f)

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Define the model
model = Sequential([
    LSTM(units=64, return_sequences=True, input_shape=(sequence_length, n_features)),
    Dropout(0.2),
    LSTM(units=64, return_sequences=True),
    Dropout(0.2),
    LSTM(units=64),
    Dropout(0.2),
    Dense(units=1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32, verbose=1)

# Make predictions on test set
y_pred = model.predict(X_test)

# Calculate the performance metrics
acc = accuracy_score(y_test, y_pred.round())
prec = precision_score(y_test, y_pred.round())
rec = recall_score(y_test, y_pred.round())
f1 = f1_score(y_test, y_pred.round())
auc = roc_auc_score(y_test, y_pred)

print("Accuracy:", acc)
print("Precision:", prec)
print("Recall:", rec)
print("F1 Score:", f1)
print("ROC AUC:", auc)


Epoch 1/50
611/611 [==============================] - 25s 25ms/step - loss: 0.5547 - accuracy: 0.6984 - val_loss: 0.3982 - val_accuracy: 0.8422
Epoch 2/50
611/611 [==============================] - 13s 21ms/step - loss: 0.3700 - accuracy: 0.8546 - val_loss: 0.3515 - val_accuracy: 0.8631
Epoch 3/50
611/611 [==============================] - 13s 21ms/step - loss: 0.3414 - accuracy: 0.8688 - val_loss: 0.3462 - val_accuracy: 0.8672
Epoch 4/50
611/611 [==============================] - 12s 20ms/step - loss: 0.3241 - accuracy: 0.8770 - val_loss: 0.3465 - val_accuracy: 0.8666
Epoch 5/50
611/611 [==============================] - 13s 21ms/step - loss: 0.3115 - accuracy: 0.8817 - val_loss: 0.3456 - val_accuracy: 0.8668
Epoch 6/50
611/611 [==============================] - 13s 21ms/step - loss: 0.3020 - accuracy: 0.8865 - val_loss: 0.3516 - val_accuracy: 0.8652
Epoch 7/50
611/611 [==============================] - 13s 21ms/step - loss: 0.2881 - accuracy: 0.8928 - val_loss: 0.3606 - val_accuracy:

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# Define the model
model = Sequential([
    LSTM(units=128, return_sequences=True, input_shape=(sequence_length, n_features)),
    Dropout(0.2),
    LSTM(units=128, return_sequences=True),
    Dropout(0.2),
    LSTM(units=64, return_sequences=True),
    Dropout(0.2),
    LSTM(units=64),
    Dropout(0.2),
    Dense(units=1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32)

# Make predictions on test set
y_pred = model.predict(X_test)
y_pred_classes = [1 if x > 0.5 else 0 for x in y_pred]

# Compute evaluation metrics
accuracy = accuracy_score(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes)
recall = recall_score(y_test, y_pred_classes)
f1 = f1_score(y_test, y_pred_classes)
roc_auc = roc_auc_score(y_test, y_pred)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_classes).ravel()
specificity = tn / (tn+fp)

# Print evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("ROC AUC Score:", roc_auc)
print("Specificity:", specificity)


Epoch 1/50
611/611 [==============================] - 43s 45ms/step - loss: 0.6631 - accuracy: 0.5858 - val_loss: 0.4810 - val_accuracy: 0.7903
Epoch 2/50
611/611 [==============================] - 22s 37ms/step - loss: 0.3939 - accuracy: 0.8397 - val_loss: 0.3807 - val_accuracy: 0.8474
Epoch 3/50
611/611 [==============================] - 23s 38ms/step - loss: 0.3440 - accuracy: 0.8667 - val_loss: 0.3603 - val_accuracy: 0.8604
Epoch 4/50
611/611 [==============================] - 25s 40ms/step - loss: 0.3264 - accuracy: 0.8762 - val_loss: 0.3470 - val_accuracy: 0.8652
Epoch 5/50
611/611 [==============================] - 26s 43ms/step - loss: 0.3109 - accuracy: 0.8844 - val_loss: 0.3569 - val_accuracy: 0.8649
Epoch 6/50
611/611 [==============================] - 25s 41ms/step - loss: 0.2997 - accuracy: 0.8890 - val_loss: 0.3538 - val_accuracy: 0.8643
Epoch 7/50
611/611 [==============================] - 24s 40ms/step - loss: 0.2865 - accuracy: 0.8945 - val_loss: 0.3627 - val_accuracy:

In [16]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Define the model architecture
def create_model(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(64, return_sequences=True, input_shape=input_shape),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(32, return_sequences=True),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(16),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

# Define the evaluation metrics
def evaluate_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    return accuracy, precision, recall, f1, roc_auc, specificity



# Define the cross-validation folds
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Iterate over the folds
fold = 5
for train_index, test_index in kf.split(X):
    # Split the data into train and test sets for this fold
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Create the model
    model = create_model(X_train.shape[1:])
    
    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Train the model
    model.fit(X_train, y_train, epochs=50, batch_size=32)
    
    # Make predictions on test set
    y_pred = model.predict(X_test)
    y_pred = np.round(y_pred)
    y_true = y_test
    
    # Evaluate the model on test set
    accuracy, precision, recall, f1, roc_auc, specificity = evaluate_metrics(y_true, y_pred)
    
    # Print the evaluation metrics
    print(f"Fold {fold}")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 score: {f1}")
    print(f"ROC AUC score: {roc_auc}")
    print(f"Specificity: {specificity}")
    print("\n")
    
    # Increase the fold counter
    fold += 1


Epoch 1/50
611/611 [==============================] - 21s 20ms/step - loss: 0.5869 - accuracy: 0.6736
Epoch 2/50
611/611 [==============================] - 14s 23ms/step - loss: 0.3895 - accuracy: 0.8499
Epoch 3/50
611/611 [==============================] - 11s 18ms/step - loss: 0.3562 - accuracy: 0.8657
Epoch 4/50
611/611 [==============================] - 11s 18ms/step - loss: 0.3395 - accuracy: 0.8720
Epoch 5/50
611/611 [==============================] - 11s 18ms/step - loss: 0.3249 - accuracy: 0.8788
Epoch 6/50
611/611 [==============================] - 11s 18ms/step - loss: 0.3163 - accuracy: 0.8835
Epoch 7/50
611/611 [==============================] - 11s 17ms/step - loss: 0.3074 - accuracy: 0.8863
Epoch 8/50
611/611 [==============================] - 11s 18ms/step - loss: 0.2998 - accuracy: 0.8908
Epoch 9/50
611/611 [==============================] - 11s 18ms/step - loss: 0.2908 - accuracy: 0.8945
Epoch 10/50
611/611 [==============================] - 11s 18ms/step - loss: 0.280

611/611 [==============================] - 10s 16ms/step - loss: 0.1499 - accuracy: 0.9455
Epoch 29/50
611/611 [==============================] - 10s 17ms/step - loss: 0.1426 - accuracy: 0.9476
Epoch 30/50
611/611 [==============================] - 10s 17ms/step - loss: 0.1396 - accuracy: 0.9495
Epoch 31/50
611/611 [==============================] - 10s 17ms/step - loss: 0.1305 - accuracy: 0.9521
Epoch 32/50
611/611 [==============================] - 11s 18ms/step - loss: 0.1282 - accuracy: 0.9542
Epoch 33/50
611/611 [==============================] - 10s 17ms/step - loss: 0.1166 - accuracy: 0.9578
Epoch 34/50
611/611 [==============================] - 10s 17ms/step - loss: 0.1086 - accuracy: 0.9600
Epoch 35/50
611/611 [==============================] - 11s 18ms/step - loss: 0.1095 - accuracy: 0.9588
Epoch 36/50
611/611 [==============================] - 11s 18ms/step - loss: 0.1042 - accuracy: 0.9610
Epoch 37/50
611/611 [==============================] - 10s 17ms/step - loss: 0.0998 -

611/611 [==============================] - 10s 17ms/step - loss: 0.3535 - accuracy: 0.8666
Epoch 4/50
611/611 [==============================] - 11s 18ms/step - loss: 0.3351 - accuracy: 0.8735
Epoch 5/50
611/611 [==============================] - 10s 17ms/step - loss: 0.3261 - accuracy: 0.8782
Epoch 6/50
611/611 [==============================] - 10s 17ms/step - loss: 0.3157 - accuracy: 0.8844
Epoch 7/50
611/611 [==============================] - 10s 17ms/step - loss: 0.3052 - accuracy: 0.8883
Epoch 8/50
611/611 [==============================] - 10s 17ms/step - loss: 0.2954 - accuracy: 0.8927
Epoch 9/50
611/611 [==============================] - 10s 17ms/step - loss: 0.2897 - accuracy: 0.8947
Epoch 10/50
611/611 [==============================] - 11s 17ms/step - loss: 0.2808 - accuracy: 0.8981
Epoch 11/50
611/611 [==============================] - 11s 18ms/step - loss: 0.2721 - accuracy: 0.9019
Epoch 12/50
611/611 [==============================] - 11s 18ms/step - loss: 0.2640 - accur

611/611 [==============================] - 11s 18ms/step - loss: 0.1477 - accuracy: 0.9450
Epoch 31/50
611/611 [==============================] - 12s 20ms/step - loss: 0.1436 - accuracy: 0.9477
Epoch 32/50
611/611 [==============================] - 12s 20ms/step - loss: 0.1362 - accuracy: 0.9486
Epoch 33/50
611/611 [==============================] - 11s 18ms/step - loss: 0.1264 - accuracy: 0.9514
Epoch 34/50
611/611 [==============================] - 11s 17ms/step - loss: 0.1250 - accuracy: 0.9535
Epoch 35/50
611/611 [==============================] - 12s 20ms/step - loss: 0.1174 - accuracy: 0.9556
Epoch 36/50
611/611 [==============================] - 11s 18ms/step - loss: 0.1168 - accuracy: 0.9554
Epoch 37/50
611/611 [==============================] - 10s 17ms/step - loss: 0.1104 - accuracy: 0.9586
Epoch 38/50
611/611 [==============================] - 11s 18ms/step - loss: 0.1061 - accuracy: 0.9598
Epoch 39/50
611/611 [==============================] - 11s 17ms/step - loss: 0.1069 -